## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [ ]:
import pandas as pd
import numpy as np
from dataclasses import dataclass
from geopy.distance import geodesic
from tqdm.auto import tqdm
from itertools import combinations
import pickle
import os
import random

os.environ["PYTHONOPTIMIZE"] = "2"

@dataclass
class City:
    name: str
    lat: float
    lon: float
    
    def __hash__(self):
        return hash(self.name)

@dataclass
class Individual:
    genome: np.ndarray
    fitness: float = None

    def __lt__(self, other):
        return self.fitness < other.fitness
    
    def __init__(self, genome: np.ndarray, fitness: float = None):
        self.genome = genome
        self.fitness = fitness

In [ ]:
CITIES = None
DIST_MATRIX = None

In [ ]:
def load_cities(country):
    global CITIES
    try:
        CITIES = np.load(f'cities/{country}.npy')
    except:
        CITIES = pd.read_csv('CITIES/'+country+'.csv', header=None, names=['city', 'lat', 'lon'])
        CITIES = [City(row.city, row.lat, row.lon) for row in CITIES.itertuples()]
        np.save(f'cities/{country}.npy', CITIES)

def load_dist_matrix(country):
    global DIST_MATRIX
    try:
        with open(f'data/dist_matrix_{country}.pkl', 'rb') as f:
            DIST_MATRIX = pickle.load(f)
    except:
        DIST_MATRIX = dict()
        for c1, c2 in combinations(CITIES, 2):
            DIST_MATRIX[(c1, c2)] = DIST_MATRIX[(c2, c1)] = geodesic((c1.lat, c1.lon), (c2.lat, c2.lon)).km
        with open(f'data/dist_matrix_{country}.pkl', 'wb') as f:
            pickle.dump(DIST_MATRIX, f)
    
def distance_fast(city1: City, city2: City):
    return DIST_MATRIX[(city1, city2)]

def distance_eucl(city1: City, city2: City):
    return ((city1.lat - city2.lat)**2 + (city1.lon - city2.lon)**2)**0.5

def distance_km(city1: City, city2: City):
    return geodesic((city1.lat, city1.lon), (city2.lat, city2.lon)).kilometers

def cost(solution, distance=distance_fast):
    prev = None
    total = 0
    for city in solution:
        if prev is None:
            prev = city
            continue
        total += distance(prev, city)
        prev = city
    total += distance(prev, solution[0])
    return total


def fitness(individual: Individual):
    return -cost(individual.genome)

def print_solution(solution):
    for city in solution:
        print(city.name, end=' -> ')
    print(solution[0].name)

# Plotting

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, LineString

def plot_solution(solution, CITIES, distance_km, cost, file=None):
    plt.figure()
    plt.tight_layout()
    city_coords = [(city.lon, city.lat) for city in solution]
    cities_gdf = gpd.GeoDataFrame(geometry=[Point(city.lon, city.lat) for city in solution])

    city_coords.append(city_coords[0])
    path = LineString(city_coords)

    world = gpd.read_file("./data/ne_110m_admin_0_countries.zip")

    # Plot world map
    fig, ax = plt.subplots(figsize=(10, 10))
    world.plot(ax=ax, color='lightgray')

    # Plot path
    gpd.GeoSeries(path).plot(ax=ax, color='blue', linewidth=1, label='Path', linestyle='-.')

    # Plot CITIES
    cities_gdf.plot(ax=ax, color='red', marker='*', markersize=7, label='Cities')

    extreme_cities = [
        max(CITIES, key=lambda city: city.lat),  # North
        min(CITIES, key=lambda city: city.lat),  # South
        max(CITIES, key=lambda city: city.lon),  # East
        min(CITIES, key=lambda city: city.lon)   # West
    ]
    temp = []
    # Labeling CITIES with their names
    for (city,horizontal,vertical) in zip(extreme_cities,['center', 'center', 'left', 'right'],['bottom', 'top', 'center', 'center']):
        if city not in temp:
            temp.append(city)
            plt.text(city.lon, city.lat, city.name, fontsize=12, ha=horizontal, va=vertical)

    plt.title(f"Traveling Salesman Problem\nTotal Distance: {cost(solution, distance=distance_km):.2f} km")

    min_x, max_x = cities_gdf.total_bounds[[0, 2]]
    min_y, max_y = cities_gdf.total_bounds[[1, 3]]
    margin_x = 0.2*(max_x - min_x)
    margin_y = 0.2*(max_y - min_y)

    ax.set_xlim([min_x-margin_x, max_x+margin_x])  
    ax.set_ylim([min_y-margin_y, max_y+margin_y])
    plt.axis('off')
    plt.legend()
    if file:
        plt.savefig(file, bbox_inches='tight')
    else:
        plt.show()

def plot_fitness(history, optimal=None, file=None, country=""):
    plt.figure()
    plt.tight_layout()
    history = history[:history.index(history[-1])+(len(history) - history.index(history[-1]))//5] 
    plt.plot(history)
    if optimal is not None:
        plt.axhline(y=optimal, color='r', linestyle=':', label='Optimal')
    plt.xlabel('Generation')
    plt.ylabel('Fitness')
    plt.title('Fitness over generations for ' + country)
    # Truncate history when it reaches the last value
    if optimal is not None:
        plt.text(len(history), history[0],f"Distance from optimal: {history[-1]/optimal * 100:.2f}" + '%', ha='right', va='top', c='gray')
    if file:
        plt.savefig(file, bbox_inches='tight')
    else:
        plt.show()

# Greedy

In [ ]:
def closest_city(start_city_index=None, p=0.1):
    visited = set()
    if start_city_index is None:
        i = random.randint(0, len(CITIES) - 1)
    else:
        i = start_city_index
    city = CITIES[i]
    visited.add(city)
    greedy_solution = np.empty(len(CITIES), dtype=object)
    greedy_solution[0] = city

    index = 1

    while len(visited) < len(CITIES):
        distances = [(c,DIST_MATRIX[(city, c)]) for c in CITIES if c not in visited and c != city]
        distances.sort(key=lambda x: x[1])

        # Choose the next city based on the distance from the current city.
        x=-1
        k = -1
        while x < p and k < len(distances) - 1:
            k += 1
            x = np.random.rand()

        city = distances[k][0]

        visited.add(city)
        greedy_solution[index] = city
        index += 1

    return greedy_solution

In [ ]:
for country in ['vanuatu','italy','russia','us', 'china']:
    load_cities(country)
    load_dist_matrix(country)

    min_cost = None
    best_greedy_solution = None
    for city in tqdm(CITIES):
        greedy_solution = closest_city(CITIES.index(city), p=0)

        if best_greedy_solution is None or cost(greedy_solution) < min_cost:
            min_cost = cost(greedy_solution)
            best_greedy_solution = greedy_solution

    print(f"Best greedy solution for {country}: {min_cost:.2f}")

# Evolutionary algorithm

In [ ]:
def parent_selection(population):
    candidates = np.random.choice(population, size=2)
    candidates = sorted(candidates, key= lambda x: x.fitness, reverse=True)
    return candidates[0]

def cycle_crossover(parent1: Individual, parent2: Individual):
    genome1 = parent1.genome
    genome2 = parent2.genome
    i, j = np.random.choice(len(genome1), size=2, replace=False)
    if i > j:
        i, j = j, i
    child = np.empty(len(genome1), dtype=object)
    child[i:j] = genome1[i:j]

    existing_cities = set(city.name for city in child[i:j])

    fill_positions = np.where(child == None)[0]
    fill_idx = 0

    for city in genome2:
        if city.name not in existing_cities:
            child[fill_positions[fill_idx]] = city
            fill_idx += 1
            existing_cities.add(city.name)
            if fill_idx == len(fill_positions):  
                break
            
    return Individual(child)


### From Michele Cazzola
def inver_over_xover(p1: Individual, p2: Individual):
    """INVER-OVER crossover"""
    genome1 = p1.genome.copy()
    genome = np.empty(genome1.shape, dtype=object)
    num_cities = p1.genome.size
    
    p1_start = np.random.randint(len(p1.genome))
    genome1 = np.roll(genome1, -p1_start)
    
    p2_start = p2.genome.tolist().index(genome1[0])
    p1_end = genome1.tolist().index(p2.genome[(p2_start+1) % num_cities])
    
    genome[0] = genome1[0]
    genome[1] = genome1[p1_end]
    
    genome[2:p1_end+1] = genome1[1:p1_end][::-1]
    genome[p1_end+1:] = genome1[p1_end+1:]
    genome = np.roll(genome, p1_start)
    
    return Individual(genome)

crossover = cycle_crossover 

def multiple_swap_mutation(p: Individual):
    genome = p.genome.copy()
    x = 0
    while x < 0.1:
        i, j = np.random.choice(len(genome), size=2, replace=False)
        genome[i], genome[j] = genome[j], genome[i]
        x = np.random.rand()
    return Individual(genome)

def scrumble_mutation(p: Individual):
    genome = p.genome.copy()
    mask = np.random.rand(len(genome)) < 0.1
    genome[mask] = np.random.permutation(genome[mask])
    return Individual(genome)

def inversion_mutation(p: Individual):
    genome = p.genome.copy()
    i, j = np.random.choice(len(genome), size=2, replace=False)
    if i > j:
        i, j = j, i
    genome[i:j] = genome[i:j][::-1]
    return Individual(genome)

mutation = inversion_mutation

In [ ]:
pop_size = {
    'vanuatu': 100,
    'italy': 100,
    'russia': 100,
    'us': 100,
    'china': 100,
}

max_gen = {
    'vanuatu': 100,
    'italy': 1000,
    'russia': 10000,
    'us': 20000,
    'china': 120000,
}

optimal = {
    'italy': 4172.76,
    'russia': 32722.5,
    'vanuatu': 1345.54,
    'us': 39016.62,
    'china': None
}

iterations = {
    'vanuatu': 1,
    'italy': 10,
    'russia': 5,
    'us': 3,
    'china': 1,
}



## Evolution

In [ ]:
for country in ['china']:

    load_cities(country)
    load_dist_matrix(country)    

    POPULATION_SIZE = pop_size[country]
    OFFSPRING_SIZE = pop_size[country] // 5
    MAX_GENERATIONS = max_gen[country]

    solution = None
    history = list()
    best_population = None
    for idx in range(iterations[country]):
        population = [Individual(closest_city(i, p=0.25)) for i in np.random.randint(0, len(CITIES), POPULATION_SIZE)] 

        temp_history = list()

        for i in population:
            i.fitness = fitness(i)

        for _ in tqdm(range(MAX_GENERATIONS)):
            offspring = list()
            
            for _ in range(OFFSPRING_SIZE):
                if np.random.random() < 0.6:
                    p = parent_selection(population)
                    o = mutation(p)
                else: 
                    i1 = parent_selection(population)
                    i2 = parent_selection(population)
                    o = crossover(i1, i2)
                offspring.append(o)
                
            for i in offspring:
                i.fitness = fitness(i)

            population.extend(offspring)
            population.sort(key=lambda i: i.fitness, reverse=True)

            population = population[:POPULATION_SIZE]

            temp_history.append(population[0].fitness)

        print(population[0].fitness)
        if solution is None or population[0].fitness > solution.fitness:
            solution = population[0]
            history = temp_history
            best_population = population
    
    try:
        with open(f'data/solution_{country}.pkl', 'rb') as f:
            s = pickle.load(f)

        if s.fitness < solution.fitness:
            with open(f'data/solution_{country}.pkl', 'wb') as f:
                pickle.dump(solution, f)
            with open(f'data/history_{country}.pkl', 'wb') as f:
                pickle.dump(history, f)
            with open(f'data/population_{country}.pkl', 'wb') as f:
                pickle.dump(best_population, f)
    except:
            with open(f'data/solution_{country}.pkl', 'wb') as f:
                pickle.dump(solution, f)
            with open(f'data/history_{country}.pkl', 'wb') as f:
                pickle.dump(history, f)
            with open(f'data/population_{country}.pkl', 'wb') as f:
                pickle.dump(best_population, f)
    print_solution(solution.genome.copy())
    plot_solution(solution.genome.copy(), CITIES, distance_km, cost)
    plot_fitness(history.copy(), -optimal[country] if optimal[country] is not None else None)
    


## Produce the output images

In [ ]:
for country in ['vanuatu', 'italy', 'russia', 'us', 'china']:
    with open(f'data/history_{country}.pkl', 'rb') as f:
        history = pickle.load(f)
    with open(f'data/solution_{country}.pkl', 'rb') as f:
        solution = pickle.load(f)
    load_cities(country)
    load_dist_matrix(country)

    print(history[-1])
    plot_fitness(history.copy(), -optimal[country] if optimal[country] is not None else None, file=f'images/fitness_{country}.png', country=country.capitalize())
    plot_solution(solution.genome.copy(), CITIES, distance_km, cost, file=f'images/TSP_{country}.png')